In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price reason market active'.split()

today = date.today()
today

datetime.date(2021, 12, 31)

### Create orders from stocks after adjust stocks in port_lite from consensus

In [12]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

25

In [13]:
sql = '''
SELECT 'B' AS trade,name, qty, buy_target, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df = pd.read_sql(sql, conlite)
df.shape

(16, 7)

In [14]:
sql = '''
INSERT INTO orders
SELECT id,'B',name, qty, buy_target, 0, reason, market
FROM stocks
WHERE status IN ('B','O')'''
rp = conlite.execute(sql)
rp.rowcount

16

In [15]:
sql = '''
SELECT 'S' AS trade,name, qty, sell_target, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')
ORDER BY name'''
df = pd.read_sql(sql, conlite)
df.shape

(7, 7)

In [16]:
sql = '''
INSERT INTO orders
SELECT id,'S',name, qty, sell_target, 0, reason, market
FROM stocks
WHERE status IN ('I','S')'''
rp = conlite.execute(sql)
rp.rowcount

7

In [17]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_orders.shape[0]

23

In [18]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/orders.csv',
 '\\Users\\User\\iCloudDrive\\orders.csv',
 '\\Users\\User\\Dropbox\\orders.csv')

In [19]:
cols = 'trade name qty price active reason market'.split()
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=False, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=False, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=False, index=False)

### After call ord-log

In [20]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [21]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9,11])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent', 'col11': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[['trans','name','spd','reason','qty','target','current','percent','active']], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

### Past 2 weeks price, max, min to determine selling, buying prices

In [63]:
name = 'ROJNA'
name

'ROJNA'

In [13]:
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT 10"""
sql = sql % name
print(sql)


SELECT *
FROM price
WHERE name = 'ROJNA'
ORDER BY date DESC
LIMIT 10


In [14]:
df_price = pd.read_sql(sql, const)
df_price.shape

(10, 7)

In [15]:
df_tmp = df_price.tail(10)
df_tmp

,name,date,price,maxp,minp,qty,opnp
0,ROJNA,2021-12-30,6.65,6.75,6.55,2145548,6.55
1,ROJNA,2021-12-29,6.55,6.60,6.55,1057098,6.55
2,ROJNA,2021-12-28,6.55,6.60,6.50,1042164,6.50
3,ROJNA,2021-12-27,6.50,6.60,6.50,1001910,6.50
4,ROJNA,2021-12-24,6.50,6.60,6.50,712333,6.60
5,ROJNA,2021-12-23,6.60,6.65,6.55,2665600,6.55
6,ROJNA,2021-12-22,6.55,6.55,6.45,1406226,6.50
7,ROJNA,2021-12-21,6.50,6.50,6.40,2166394,6.40
8,ROJNA,2021-12-20,6.40,6.75,6.35,10048349,6.70
9,ROJNA,2021-12-17,6.80,6.85,6.60,7834657,6.65


In [16]:
df_tmp.maxp.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,maxp
6.6,40.00%
6.75,20.00%
6.65,10.00%
6.55,10.00%
6.5,10.00%
6.85,10.00%


In [17]:
df_tmp.minp.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,minp
6.55,30.00%
6.5,30.00%
6.45,10.00%
6.4,10.00%
6.35,10.00%
6.6,10.00%


In [18]:
df_tmp.maxp.max(),df_tmp.minp.min()

(6.85, 6.35)

In [19]:
100000/10.4

9615.384615384615